In [2]:
import MDRMF as mf

In [3]:
df = mf.MoleculeLoader("10K.csv","SMILES","r_i_docking_score").df

In [31]:
# featurizer.py

import pandas as pd
import numpy as np
from typing import Optional
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprintAsBitVect
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem import MACCSkeys
from rdkit import DataStructs


class Featurizer:
    """
    A class to featurize molecules in a DataFrame.
    """
    def __init__(self, df: pd.DataFrame = None, mol_col: str = 'molecules') -> None:
        """
        Initializes the Featurizer with a DataFrame and the name of the column containing molecules.

        Args:
            df (pd.DataFrame): The DataFrame to featurize.
            mol_col (str): The name of the column containing molecules to featurize.
        """
        self.df = df
        self.mol_col = mol_col
        self.smi_col = 'SMILES'
        self.features = None

    def featurize(self, method: str, **kwargs) -> None:
        """
        Featurizes the molecules in the DataFrame using the specified method and stores the features separately.

        Args:
            method (str): The featurization method to use. Supported methods are 'morgan' and 'topological'.
            **kwargs: Additional keyword arguments to pass to the featurization method.
        """
        if method == 'morgan':
            # self.df['features'] = self.df[self.mol_col].apply(lambda mol: self._convert_to_np_array(AllChem.GetMorganFingerprintAsBitVect(mol, **kwargs)))
            features_gen = tuple(self.df[self.mol_col].apply(lambda mol: self._convert_to_np_array(AllChem.GetMorganFingerprintAsBitVect(mol, **kwargs))))
        elif method == 'topological':
            features_gen = tuple(self.df[self.mol_col].apply(lambda mol: self._convert_to_np_array(FingerprintMols.FingerprintMol(mol, **kwargs))))
            # self.df['features'] = self.df[self.mol_col].apply(lambda mol: self._convert_to_np_array(FingerprintMols.FingerprintMol(mol, **kwargs)))
        elif method == 'MACCS':
            features_gen = self.df[self.mol_col].apply(lambda mol: MACCSkeys.GenMACCSKeys(mol, **kwargs))
        else:
            raise ValueError(f"Unsupported featurization method: {method}")

        self.features = np.vstack(features_gen)

        return self.features

    def _convert_to_np_array(self, bit_vect) -> np.ndarray:
        """
        Converts an RDKit explicit bit vector to a numpy array.

        Args:
            bit_vect: The bit vector to convert.

        Returns:
            np.ndarray: The converted numpy array.
        """
        np_array = np.zeros((1, bit_vect.GetNumBits()), dtype=np.int8)
        DataStructs.ConvertToNumpyArray(bit_vect, np_array)
        return np_array
    
    def get_df(self):
        """
        Returns:
            The DataFrame
        """
        return self.df

    def get_features(self):
        """
        Returns the 2D numpy array of featurized molecules.

        Returns:
            np.ndarray: The featurized molecules.
        """
        if self.features is not None:
            return self.features
        else:
            print("No features available. Please run the featurize method first.")

    def inspect_features_by_smiles(self, smiles: str) -> Optional[np.ndarray]:
        """
        Inspects the features for a specific molecule based on its SMILES representation.

        Args:
            smiles (str): The SMILES string for the molecule to inspect.

        Returns:
            np.ndarray: The feature vector for the molecule, or None if the molecule is not found.
        """
        index = self.df[self.df[self.smi_col] == smiles].index
        if not index.empty:
            fingerprint = self.df['features'][index[0]]
            return fingerprint
        else:
            print(f"No molecule with SMILES {smiles} found in the DataFrame.")
            return None

In [32]:
featurizer = Featurizer(df, mol_col="molecules")

In [33]:
feats = featurizer.featurize("MACCS")

In [39]:
feats[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [13]:
df

,Unnamed: 0,i_i_glide_lignum,r_i_docking_score,SMILES,molecules
0,0,13477,-13.126800,C[C@@H](NC(=O)N1C[C@H](c2ccccc2)[C@H]2COCC[C@H...,<rdkit.Chem.rdchem.Mol object at 0x000001BB295...
1,1,516,-12.662900,O=C(Nc1cccc(C(=O)N2CCC(c3c[nH]c4ncccc34)CC2)c1...,<rdkit.Chem.rdchem.Mol object at 0x000001BB295...
2,2,9425,-12.487000,Cc1nn(-c2ccccc2)c2nc(C(=O)N3CCC([C@H]4C(=O)Nc5...,<rdkit.Chem.rdchem.Mol object at 0x000001BB295...
3,3,14512,-12.483500,Cc1cc(C)cc(C(=O)N2CCC[C@H](C(=O)NCc3cccc([C@@]...,<rdkit.Chem.rdchem.Mol object at 0x000001BB295...
4,4,9615,-12.478500,CS(=O)(=O)c1ccc(F)c(C(=O)Nc2ccc(-c3nc(-c4ccccc...,<rdkit.Chem.rdchem.Mol object at 0x000001BB295...
...,...,...,...,...,...
9893,9893,2451,-0.560302,O=C([O-])C(F)(F)F,<rdkit.Chem.rdchem.Mol object at 0x000001BB294...
9894,9894,7813,-0.384887,Cc1cnn(C)c1S(=O)(=O)NC[C@@H](Cc1ccccn1)C(=O)[O-],<rdkit.Chem.rdchem.Mol object at 0x000001BB294...
9895,9895,11442,-0.269109,Cn1cccc(C(=O)N2CCN(C3CC3)c3ccc(Cl)cc32)c1=O,<rdkit.Chem.rdchem.Mol object at 0x000001BB294...
9896,9896,13971,-0.150473,O=C([O-])[C@H]1CC[C@@H](C(=O)N2CCCc3ccccc32)CC1,<rdkit.Chem.rdchem.Mol object at 0x000001BB294...


In [23]:
def convert_to_np_array(bit_vect) -> np.ndarray:
    """
    Converts an RDKit explicit bit vector to a numpy array.

    Args:
        bit_vect: The bit vector to convert.

    Returns:
        np.ndarray: The converted numpy array.
    """
    np_array = np.zeros((1, bit_vect.GetNumBits()), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(bit_vect, np_array)
    return np_array

In [27]:
fings = df['molecules'].apply(lambda mol: MACCSkeys.GenMACCSKeys(mol))

In [28]:
type(fings)

pandas.core.series.Series